In [1]:
from vk_api import VkApi
from math import ceil
from tqdm import tqdm, trange

with open('token.txt', 'r') as tokens:
  token = tokens.readline()
  vk = VkApi(token = token)

pub_name = 'poiskmemow'
max_items = 100

In [2]:
count = vk.method("wall.get", {'domain': pub_name, 'count': 1})['count']
api_calls_count = ceil(count / max_items)
print(count, api_calls_count)

3435 35


In [3]:
posts_data = []
for i in trange(api_calls_count):
  request_data = {'domain': pub_name, 'count': max_items, 'offset': i * max_items}
  posts_data += vk.method("wall.get", request_data)['items']

100%|██████████| 35/35 [00:37<00:00,  1.08s/it]


In [13]:
import re

def format_text(text):
  url_re = r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,4}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)'
  src_url = f'([Ии]сточник.*?)?( {url_re})+'
  tags_re = r'#.+?@\S+'
  # clear text
  text = text.replace('\n', ' ').replace('\r', ' ') # remove newlines
  text = re.sub(src_url, '', text) # remove source links
  text = re.sub(url_re, '', text) # remove urls
  text = re.sub(tags_re, '', text) # remove hashtags
  text = text.replace('#', '') # convert hashtags to words
  text = ' '.join(text.split()).strip() # remove double spaces
  return text

In [21]:
class Post:
  def __init__(self, json):
    self.json, self.id = json, json['id']
    self.text, self.photos = self._parse_post(json)

  def _parse_post(self, json):
    text = format_text(json['text'])
    photos = []
    if 'attachments' in json.keys():
      for attachment in json['attachments']:
        if attachment['type'] == 'photo':
          max_src = max(attachment['photo']['sizes'], key=lambda x: x['width'] * x['height'])['url']
          photos.append(max_src)
    return text, photos
  
  def __str__(self):
    return f'Post(id={self.id}, text={self.text}, photos={self.photos})'

In [22]:
posts = [Post(post) for post in posts_data]
print("all posts:", len(posts))

posts = [post for post in posts if post.photos and post.text]
print("without images:", len(posts))

all posts: 3435
without images: 3404


In [23]:
with open('only_text.txt', 'w', encoding="utf-8") as f:
  for post in posts:
    f.write(f'{post.text}\n')

In [ ]:
from random import choice
print(choice(posts))